# **Модуль А:** Анализ и предобработка данных

**Краткое описание задания:** в рамках данного модуля участникам необходимо подготовить набор данных для обучения дублирующей нейронной сети, а также ознакомится с предоставленными организаторами API.

## Импортирование библиотек

In [6]:
# библиотека
import json
import time

import requests
import base64

import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image

from sklearn.model_selection import train_test_split

from config import API_KEY, API_SECRET

from transformers import AutoTokenizer, AutoModelForSequenceClassification

Для того чтобы скачать сертификат, используется следующая команда:

```bash
curl -k "https://gu-st.ru/content/Other/doc/russian_trusted_root_ca.cer" > russian_ca.cer
```

In [18]:
import os
import ssl
import urllib.request
from certifi import where as certifi_where

def add_trusted_certificate(cert_url, cert_filename=None):
    """
    Загружает и добавляет SSL-сертификат в доверенные сертификаты Python
    
    :param cert_url: URL сертификата
    :param cert_filename: Имя файла для сохранения (опционально)
    :return: Путь к файлу с сертификатами
    """
    # Определяем путь к файлу сертификатов Python
    certifi_path = certifi_where()
    
    # Если имя файла не указано, используем временное
    if cert_filename is None:
        cert_filename = "downloaded_certificate.cer"
    
    try:
        # Скачиваем сертификат (с отключенной проверкой SSL)
        context = ssl._create_unverified_context()
        with urllib.request.urlopen(cert_url, context=context) as response:
            cert_data = response.read()
        
        # Сохраняем сертификат в файл
        with open(cert_filename, 'wb') as cert_file:
            cert_file.write(cert_data)
        
        # Добавляем сертификат в доверенные
        with open(certifi_path, 'ab') as trusted_certs:
            trusted_certs.write(b"\n")  # Добавляем пустую строку перед новым сертификатом
            trusted_certs.write(cert_data)
        
        print(f"Сертификат успешно добавлен в: {certifi_path}")
        return certifi_path
    
    except Exception as e:
        print(f"Ошибка при добавлении сертификата: {str(e)}")
        return None

# Пример использования:

cert_url = "https://gu-st.ru/content/Other/doc/russian_trusted_root_ca.cer"
add_trusted_certificate(cert_url, "russian_ca.cer")

Сертификат успешно добавлен в: d:\Helper\MLBazyak\chemp\.venv\Lib\site-packages\certifi\cacert.pem


'd:\\Helper\\MLBazyak\\chemp\\.venv\\Lib\\site-packages\\certifi\\cacert.pem'

In [1]:
from transformers import pipeline

In [ ]:

classifier = pipeline("zero-shot-classification",
                      model="symanto/xlm-roberta-base-snli-mnli-anli-xnli")


In [24]:
with open("internet_qa.json", encoding='utf-8') as f:
    data = json.load(f)
    print(data[0]['tags'])
    tags = list(set([item["tags"][0] for item in data]))

print(len(data))

print(len(tags))

['базовое']
43
30


In [ ]:
tags = []
for item in data:
    print(len(item))
    for i in range(len(item)):
        print(i)
        print(item)
        print(item[i+1])
        # tags.append(item[i]) 

tags = list(set(tags))
print(len(tags))

3
0
{'question': 'Почему не работает интернет?', 'answer': 'Проверьте: 1) Подключение кабеля к роутеру 2) Индикаторы на роутере 3) Наличие баланса у провайдера', 'tags': ['базовое']}


KeyError: 1

In [25]:
# 2. Загружаем модель
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(tags),
    problem_type="single_label_classification"  # Для мультиклассовой классификации
)

print(f"Модель загружена для {len(tags)} категорий: {tags}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Модель загружена для 30 категорий: ['lan', 'vpn', 'dns', 'ремонт', 'стабильность', 'диагностика', 'роутер', 'windows', 'wi-fi', 'аутентификация', 'гроза', 'браузер', 'подключение', 'базовое', 'оптимизация', 'voip', 'телефон', 'покрытие', 'ошибка', 'перебои', 'срочное', 'скорость', 'настройка', 'аппаратное', 'мобильное', 'переезд', 'безопасность', 'провайдер', 'dhcp', 'ethernet']
